<h1><center>Computing a killPlacePerc</center></h1>
To compute statistics match- and group-wise, it could make sense to compute a killPlacePerc similar to winPlacePerc.

We do so below and notice along the way that:
1. Sometimes, poor players have a killPlace worse than the total number of players in their matchId. Quite a feat!
2. How the killPlace value for ties is computed may lead to interesting information on winPlacePerc

According to <a href="https://www.kaggle.com/c/pubg-finish-placement-prediction/data">the competition data description</a>:
killPlace - Ranking in match of number of enemy players killed

Tl;dr = killPlacePerc = (killPlace-1)/(matchSize-1) where:
- matchSize is the # of Ids that share the same matchId
-  killPlace is computed by rescaling the existing killPlace between 0 and matchSize-1

Recomputing the killPlace the Ids of a same matchId by their number of kills according to the min method of the <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rank.html">pandas rank function</a>  would lose the existing information, if any, in the existing ordinal ordering of killPlace. We thus prefer to rescale the existing killPlace while maintaining their order and use then pandas rank with the first function.

The killPlace tie-breaking mechanism could be further reverse-engineered still...

We could also add the killPerc feature, which is close to killPlacePerc but not quite the same: killPerc = kills/(matchSize - 1)
We leave it to you to check which feature is more useful to predict the target. Enjoy.

In [ ]:
import gc

import numpy as np 
import pandas as pd 

In [ ]:
train = pd.read_csv("../input/train_V2.csv")
test = pd.read_csv("../input/test_V2.csv")

In [ ]:
#Checking that there are no overlap between the train and test sets matchIds
set.intersection(set(train['matchId'].unique()),set(test['matchId'].unique()))

In [ ]:
#Remowing the sample of the train set that has winPlacePerc == np.nan
train.dropna(subset=['winPlacePerc'],inplace=True)

In [ ]:
test['winPlacePerc'] = np.nan

In [ ]:
df = pd.concat([train,test],axis=0)
df.drop(df.columns.difference(['Id','matchId','groupId','kills','killPlace','maxPlace','numGroups','winPlacePerc']),axis=1,inplace=True)
del train
del test
gc.collect();

In [ ]:
#Adding the matchSize feature i.e. the # of Ids that share the same matchId
df = pd.merge(df,df.groupby('matchId')[['Id']].size().to_frame('matchSize').reset_index(),on='matchId',how='left')

In [ ]:
#Adding a flag for the test set
df['isTest'] = df['winPlacePerc'].isnull().astype(np.int8)

In [ ]:
#Checking that no matchSize is less or equal to 1
df[['matchSize']].describe()

In [ ]:
#Adding a killPlacePerc feature
df['killPlacePerc'] = (df['matchSize'] - df['killPlace']) / (df['matchSize'] - 1)

In [ ]:
#Checking if our killPlacePerc is between 0 and 1
df[['killPlacePerc']].describe()

In [ ]:
#Investigating the negative killPlacePerc
df[df['killPlacePerc'] < 0].sort_values(by='matchSize').head()

In [ ]:
#In the first row above, we can see that killPlace == 8 although matchSize == 7, let's zoom on this matchId
df[df['matchId'] == '8799301e853202'].sort_values(by='killPlace')

In [ ]:
#We can see that there is no killPlace == 3 in the matchId above 
#We can also see that all players with 0 kills are ranked although they are tied, 
#And that this tiebreak ranking seems to be linked to winPlacePerc
#Let us check that this is indeed the case, namely that:
#For every matchId, Ids that have the same number of kills have their killPlace ranked by decreasing winPlacePerc

In [ ]:
#Let us first check below that there aren't any ties for killPlace
df_noties = df.groupby(['matchId','killPlace'])[['Id']].nunique()
df_noties.max()

In [ ]:
#Let's now check that all tiebreaks in the train dataset are according to winPlacePerc
df_tiebreak = df[df['isTest'] !=1].sort_values(by=['matchId','kills','killPlace'],ascending=[False,False,True])
df_tiebreak = df_tiebreak.groupby(['matchId','kills','killPlace'])[['winPlacePerc']].mean().groupby(['matchId','kills'])[['winPlacePerc']].diff()
df_tiebreak.sort_index(ascending=[False,False,True],inplace=True)
df_tiebreak.head(15)

In [ ]:
df_tiebreak.dropna(inplace=True)
df_tiebreak.max()

In [ ]:
#Now that we have proven that the tiebreaking is by non-increasing winPlacePerc,
#Let us investigate another matchId
df[df['matchId'] == '58eb66dd8a0764'].sort_values(by='killPlace')

In [ ]:
# On this latter example we see that killPlacePerc tie-breaks for Ids in the same groupId and with the same number of kills 
# This could be misleading for winPlacePerc prediction: we prefer to keep the tie.

In [ ]:
# So let us define a more adequate killPlacePerc:
df_killPlace = df.groupby(['matchId','Id'])[['kills','killPlace']].mean()
df_killPlace.sort_values(by=['matchId','kills','killPlace'],ascending=[False,False,True],inplace=True)
df_killPlace = df_killPlace.groupby('matchId')['kills'].rank(method='first',ascending=False).to_frame('killPlacePerc').reset_index()

In [ ]:
df.drop('killPlacePerc',axis=1,inplace=True)
df = df.merge(df_killPlace,on=['matchId','Id'],how='left')

In [ ]:
df['killPlacePerc'] = df.groupby(['matchId','kills','groupId'])[['killPlacePerc']].transform(np.mean)
df['killPlacePerc'] = (df['matchSize'] - df['killPlacePerc'])/ (df['matchSize'] - 1)

In [ ]:
#Let's check the two matchIds we zoomed on hereabove
df.loc[df['matchId'].isin(['8799301e853202','58eb66dd8a0764']),['matchId','groupId','Id','killPlace','kills','winPlacePerc','killPlacePerc']].sort_values(by=['matchId','killPlace'],ascending=[False,True])